In [2]:
%matplotlib notebook

In [1]:
#Libraries
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, Angle
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

import matplotlib.pyplot as plt
import numpy as np
import requests
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Polygon
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import pandas as pd

from tqdm import tqdm
import requests

from pyvo.dal import sia  #new! for what? Simple Image Access


In [2]:
telemetry1 = pd.read_csv('decam_telemetry_20230817.csv.gz')
streaks1 = pd.read_csv('streaks_augmented_20230817.csv.gz')

In [3]:
telemetry1

,EXPNUM,BAND,TELESCOP,OBSERVAT,INSTRUME,OBS-LONG,OBS-LAT,OBS-ELEV,TELEQUIN,TIMESYS,DATE-OBS,TIME-OBS,DARKTIME,EXPTIME,RA,DEC,AZ,ZD
0,140520,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:25:35.429106,00:25:35.429106,2.10353,1.0,22:08:24.000,-32:59:17.599,116.2500,7.09
1,140521,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:29:50.667562,00:29:50.667562,2.09597,1.0,22:08:23.970,-32:55:57.799,118.9300,6.25
2,140522,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:46:16.576959,00:46:16.576959,31.09820,30.0,18:03:35.240,-30:02:00.600,254.2700,50.20
3,140523,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:47:44.370016,00:47:44.370016,31.09300,30.0,18:03:35.250,-30:02:00.802,254.1500,50.50
4,140524,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:49:47.211636,00:49:47.211636,31.09170,30.0,18:03:35.240,-30:02:00.802,253.9900,50.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506513,1228730,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:42:53.917964,08:42:53.917964,81.10140,80.0,03:30:36.090,-28:05:58.200,97.1620,36.05
506514,1228731,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:44:44.058111,08:44:44.058111,71.12170,70.0,03:30:36.079,-28:05:58.200,97.0003,35.66
506515,1228732,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:46:22.681357,08:46:22.681357,71.10870,70.0,03:30:36.090,-28:05:58.200,96.8515,35.30
506516,1228733,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:48:02.549261,08:48:02.549261,91.09020,90.0,03:30:36.112,-28:05:58.099,96.7036,34.94


In [4]:
streaks1

,filename,rnum,ra_1,ra_2,ra_3,ra_4,dec_1,dec_2,dec_3,dec_4,expnum,ccdnum,band
0,D00145425_g_c03_r4148p01_hstreak.fits,3,1.159330,1.160919,1.164310,1.162720,-28.724241,-28.724212,-28.857155,-28.857183,145425,3,g
1,D00145425_g_c06_r4148p01_hstreak.fits,1,0.986677,0.986711,0.644209,0.644186,-29.032018,-29.034283,-29.037208,-29.034944,145425,6,g
2,D00145425_g_c09_r4148p01_hstreak.fits,1,0.452852,0.452942,0.109842,0.109782,-29.193042,-29.199694,-29.203265,-29.196615,145425,9,g
3,D00145425_g_c15_r4148p01_hstreak.fits,1,0.286716,0.286812,0.630816,0.630741,-29.363308,-29.358405,-29.362739,-29.367644,145425,15,g
4,D00145425_g_c16_r4148p01_hstreak.fits,1,0.985852,0.987446,0.987211,0.985615,-29.216748,-29.216752,-29.366385,-29.366381,145425,16,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...
879115,D01223535_g_c05_r6224p01_hstreak.fits,1,323.500201,323.502653,323.444690,323.442238,-17.828499,-17.830897,-17.884254,-17.881855,1223535,5,g
879116,D01223535_g_c31_r6224p01_hstreak.fits,1,324.355544,324.369908,324.372363,324.357994,-18.392109,-18.391941,-18.541070,-18.541235,1223535,31,g
879117,D01223535_g_c56_r6224p01_hstreak.fits,1,322.914589,322.934330,322.849385,322.829651,-19.193809,-19.210870,-19.298406,-19.281342,1223535,56,g
879118,D01223535_g_c58_r6224p01_hstreak.fits,1,323.771850,323.783894,323.835739,323.823696,-19.214779,-19.204086,-19.255903,-19.266598,1223535,58,g


In [13]:
telemetry1.columns

Index(['EXPNUM', 'BAND', 'TELESCOP', 'OBSERVAT', 'INSTRUME', 'OBS-LONG',
       'OBS-LAT', 'OBS-ELEV', 'TELEQUIN', 'TIMESYS', 'DATE-OBS', 'TIME-OBS',
       'DARKTIME', 'EXPTIME', 'RA', 'DEC', 'AZ', 'ZD'],
      dtype='object')

### July 22
Take each RA and DEC to the list :p

In [195]:
telemetry1.loc[telemetry1['EXPNUM']==417465]

,EXPNUM,BAND,TELESCOP,OBSERVAT,INSTRUME,OBS-LONG,OBS-LAT,OBS-ELEV,TELEQUIN,TIMESYS,DATE-OBS,TIME-OBS,DARKTIME,EXPTIME,RA,DEC,AZ,ZD
135699,417465,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2015-03-03 01:32:35.883325,01:32:35.883325,111.097,110.0,03:58:06.870,-26:54:36.396,260.3184,46.33


In [197]:
streaks1.loc[streaks1['expnum']==454967]

,filename,rnum,ra_1,ra_2,ra_3,ra_4,dec_1,dec_2,dec_3,dec_4,expnum,ccdnum,band
229489,D00454967_g_c03_r4639p02_hstreak.fits,1,10.051430,10.060538,10.052339,10.043210,-43.014697,-43.014972,-43.164544,-43.164263,454967,3,g
229490,D00454967_g_c15_r4639p02_hstreak.fits,2,9.494397,9.494409,9.081123,9.081124,-43.508458,-43.509848,-43.511022,-43.509632,454967,15,g
229491,D00454967_g_c49_r4639p02_hstreak.fits,1,10.363983,10.364030,9.944754,9.944721,-44.473971,-44.476235,-44.480788,-44.478522,454967,49,g


In [177]:
row_958013 = telemetry1.loc[telemetry1['EXPNUM']==1074509]
ra = row_958013['RA']
dec = row_958013['DEC']
print(ra.values, dec.values)

['08:44:00.118'] ['00:59:59.500']


In [178]:
coord = SkyCoord(ra=row_958013['RA'].values[0], dec=row_958013['DEC'].values[0], unit=u.deg)
print(coord.icrs.ra.value, coord.icrs.dec.value)

8.73336611111111 0.9998611111111111


| EXPNUM  |      RA      |         DEC      |
|---------|--------------|------------------|
| 958013  | 9.85464111111111 |   -8.973971944444445  |
| 1220019  | 0.524993888888889 |  -43.008028055555556 |
| 1011967  |  23.909376944444443 | -10.335555000000001  |
| 1103448  | 1.5165919444444445  | -14.293888055555556  |
| 1114861  | 21.508803055555557  | -33.23194388888889 |
| 1218180 | 0.6334330555555555  |  -43.998 |
| 1157371  | 6.037903055555556  | -22.690306111111113  |
| 949790   | 0.9333688888888889   |  -73.04222  | 
| 1170668   | 3.2705688888888886  |  -16.869083055555556   |
| 1106509   | 14.926321944444444  |  -11.538083055555555   |
| 1033925   | 0.433375  |   20.49169388888889  |
|  1132514  | 18.259083055555557  |  -53.777028055555554   |
|  1037961  | 19.66654388888889  |  -60.109831944444444   |
|   1226457 |  12.71288111111111 |  -20.17613888888889   |
|  1085358  | 10.547926944444445  |  -58.679386944444445   |
|  1040356  | 2.377368888888889  |   -7.5966661111111105  |
| 1061946   |  1.3943919444444444 |  2.7987219444444444   |
|  1165186  | 3.784631111111111  | -15.61825    |
|  1161606   | 7.720708055555556  | -1.098806111111111    |
|  1180245  |  9.951374999999999 |  -44.636028055555556   |
|  1021815  |  18.47583 |  -28.298776944444445   |
|  1049047   | 4.500743888888889  |   -7.797166111111111  |
| 922179   |  1.102393888888889 | -73.89205305555556    |
|  1043912  | 1.0674919444444444  | -72.28147    |
| 1165831  | 2.2984638888888886  |  -4.929278055555556   |
|  1184323  | 11.692716944444445  |  -12.34675   |
| 1123921   | 22.84334388888889  |  10.92811111111111   |
| 988805   | 14.243046944444444   |   1.5880561111111113  |
| 1161999  |  7.754693888888889    |  -26.249833055555555  |
| 1134857  | 21.494933055555556   | -27.814278055555555   |
|  1083266  | 6.10438611111111   | -25.789388055555555   |
| 1132532   |  18.650391944444443  | -43.35552805555556   |
|  1149389  | 3.230643888888889   |  -14.602528055555554  |
|  1057378  | 1.1175780555555557   |  -0.029471944444444443   |
|  1221574  |  3.233  |  -16.63186111111111  |
| 1141226   |  0.920286111111111  | 2.9979719444444446   |
|  1138498  |  0.6333361111111111  | -43.99766694444445   |
|  1131667  | 23.32035611111111   |  -2.86875  |
| 1165946   | 6.2646138888888885   | -21.691806111111113   |
|  1196162  | 10.010781111111111   |  2.2106669444444447  |
|  1089625  | 10.012036111111112   | -6.687861111   |
| 931641   |  5.470513888888889  |  -9.331138888888889  |
| 1162659   | 3.230525   |  -14.602971944444445  |
|  1072590  | 4.211   | -8.296943888888888   |
|  1018216  | 0.8389388888888889   | -37.643053888888886   |
|  1161471  |  2.298386111111111  |  -4.92944388888889  |
| 1087227   |  9.156336111111111  | 0.2474169444   |
|  941429  |  4.321793888888888   |  -7.503028055555555  |
| 1068909   | 1.9338111111111111   |  -14.363388055555555  |
| 1117661   |  22.460491944444442  |  -30.559306111111113 |
|  1134933  | 0.307675 | -12.571638888888888|
|  1189051  | 9.177313888888888   | -10.572971944444443   |
| 1074509   |  8.73336611111111 | 0.9998611111111111   |
| 1076136  |    3.4774611111111113 | -55.76802611111111   |
| 1129877    |   20.886683055555554 | -15.339028055555556    |
| 1126795  |   2.632893888888889 | -1.8876669444444445   |
|  938406  |   2.4301530555555555 | -4.597471944444444   |
| 1140353   |    4.001218888888889 | -8.297528055555555  |
|  1099186  |   9.202533055555556 | 15.89941611111111    |
| 1069495   |   4.853463888888888 | -52.994886944444445  |
| 1160135  | 3.230613888888889 | -14.60275       |
|  1182026  |  9.699111111111112 | -39.71608305555556    |
| 1160285   |   10.718546944444444 | -6.008943888888889    |
| 1149259    |   23.125166944444445 | 0.07313888888888889    | 


